In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)

2.0.0
sys.version_info(major=3, minor=7, micro=4, releaselevel='final', serial=0)
matplotlib 3.1.1
numpy 1.16.5
pandas 0.25.2
sklearn 0.21.3
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [4]:
# 常量
# 1. 索引操作
t = tf.constant([[1.,2.,3.],[4.,5.,6.]])
print(t)
print(t[:,1:]) # 取出第二列以后的数值
print(t[...,1]) # 取出第二列

tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[2. 3.]
 [5. 6.]], shape=(2, 2), dtype=float32)
tf.Tensor([2. 5.], shape=(2,), dtype=float32)


In [5]:
# 算子操作
print(t+10)
print(tf.square(t))
print(t@tf.transpose(t))

tf.Tensor(
[[11. 12. 13.]
 [14. 15. 16.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[ 1.  4.  9.]
 [16. 25. 36.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[14. 32.]
 [32. 77.]], shape=(2, 2), dtype=float32)


In [6]:
# numpy conversion tensor
print(t.numpy())
print(np.square(t))
np_t = np.array([[1.,2.,3.],[4.,5.,6.]])
print(tf.constant(np_t))

[[1. 2. 3.]
 [4. 5. 6.]]
[[ 1.  4.  9.]
 [16. 25. 36.]]
tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float64)


In [7]:
# 0 维 scalars
t = tf.constant(2.53)
print(t.numpy)
print(t.shape)

<bound method _EagerTensorBase.numpy of <tf.Tensor: id=17, shape=(), dtype=float32, numpy=2.53>>
()


In [8]:
# string
t = tf.constant("cafe")
print(t)
print(tf.strings.length(t))
print(tf.strings.length(t,unit="UTF8_CHAR"))
print(tf.strings.unicode_decode(t,"UTF8"))

tf.Tensor(b'cafe', shape=(), dtype=string)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor([ 99  97 102 101], shape=(4,), dtype=int32)


In [11]:
# string array
t = tf.constant(["cafe","coffee","咖啡"])
print(t)
print(tf.strings.length(t))
print(tf.strings.length(t,unit="UTF8_CHAR"))
r = tf.strings.unicode_decode(t,"UTF8")
print(r)

tf.Tensor([b'cafe' b'coffee' b'\xe5\x92\x96\xe5\x95\xa1'], shape=(3,), dtype=string)
tf.Tensor([4 6 6], shape=(3,), dtype=int32)
tf.Tensor([4 6 2], shape=(3,), dtype=int32)
<tf.RaggedTensor [[99, 97, 102, 101], [99, 111, 102, 102, 101, 101], [21654, 21857]]>


In [13]:
# ragged tensor
r = tf.ragged.constant([[11,12],[21,22,23],[],[41]])
# index op
print(r)
print(r[1])
print(r[1:3])

<tf.RaggedTensor [[11, 12], [21, 22, 23], [], [41]]>
tf.Tensor([21 22 23], shape=(3,), dtype=int32)
<tf.RaggedTensor [[21, 22, 23], []]>


In [16]:
# ops on ragged tensor
r2 = tf.ragged.constant([[51,55],[],[71]])
print(tf.concat([r,r2],axis=0)) # axis=0 行拼接
# print(tf.concat([r,r2],axis=1)) # 错误

<tf.RaggedTensor [[11, 12], [21, 22, 23], [], [41], [51, 55], [], [71]]>


In [17]:
r3 = tf.ragged.constant([[13,14],[15],[],[42,43]])
print(tf.concat([r,r3],axis=1))

<tf.RaggedTensor [[11, 12, 13, 14], [21, 22, 23, 15], [], [41, 42, 43]]>


In [18]:
# ragged tensor 变成 tensor
print(r.to_tensor())

tf.Tensor(
[[11 12  0]
 [21 22 23]
 [ 0  0  0]
 [41  0  0]], shape=(4, 3), dtype=int32)


In [20]:
# sparse tensor
s = tf.SparseTensor(indices=[[0,1],[1,0],[2,3]],values=[1.,2.,3.],dense_shape=[3,4])
print(s)
print(tf.sparse.to_dense(s))


SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
tf.Tensor(
[[0. 1. 0. 0.]
 [2. 0. 0. 0.]
 [0. 0. 0. 3.]], shape=(3, 4), dtype=float32)


In [22]:
# ops on sparse tensors

s2 = s*2.0
print(s2)
try:
    s3 = s + 1 # sparse tensors 不支持加法
except TypeError as ex:
    print(ex)
    
s4 = tf.constant([[10.,20.],
                [30.,40.],
                [50.,60.],
                [70.,80.]])
# sparse tensor 和 tensor 相乘
print(tf.sparse.sparse_dense_matmul(s,s4))

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([2. 4. 6.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
unsupported operand type(s) for +: 'SparseTensor' and 'int'
tf.Tensor(
[[ 30.  40.]
 [ 20.  40.]
 [210. 240.]], shape=(3, 2), dtype=float32)


In [24]:
# sparse tensor
s5 = tf.SparseTensor(indices=[[0,2],[0,1],[2,3]],values=[1.,2.,3.],dense_shape=[3,4])
print(s5)
print(tf.sparse.to_dense(s5)) # 索引必须事排好须的


SparseTensor(indices=tf.Tensor(
[[0 2]
 [0 1]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))


InvalidArgumentError: indices[1] = [0,1] is out of order [Op:SparseToDense]

In [26]:
# 重新排序
s6 = tf.sparse.reorder(s5)
print(tf.sparse.to_dense(s6))

tf.Tensor(
[[0. 2. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 3.]], shape=(3, 4), dtype=float32)


In [27]:
# Variables
v = tf.Variable([[1.,2.,3.],[4.,5.,6.]])
print(v)
print(v.value()) # 变成tensor
print(v.numpy()) # 变成 常量

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>
tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
[[1. 2. 3.]
 [4. 5. 6.]]


In [29]:
# assign value
v.assign(2*v)
print(v.numpy())
v[0,1].assign(42)
print(v.numpy())
v[1].assign([7.,8.,9.])
print(v.numpy())

[[ 4.  8. 12.]
 [16. 20. 24.]]
[[ 4. 42. 12.]
 [16. 20. 24.]]
[[ 4. 42. 12.]
 [ 7.  8.  9.]]


In [30]:
try:
    v[1] = [7.,8.,9.]
except TypeError as ex:
    print(ex)

'ResourceVariable' object does not support item assignment
